In [ ]:
####################################
#ENVIRONMENT SETUP

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [ ]:
#Additional Libraries
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

In [ ]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [ ]:
def GetPlottingDirectory(plotFileName, plotType):
    plottingDirectory = mainCodeDirectory=os.path.join(mainDirectory,"Code","PLOTTING")
    
    specificPlottingDirectory = os.path.join(plottingDirectory, plotType, 
                                             f"{ModelData.res}_{ModelData.t_res}_{ModelData.Nz_str}nz")
    os.makedirs(specificPlottingDirectory, exist_ok=True)

    plottingFileName=os.path.join(specificPlottingDirectory, plotFileName)

    return plottingFileName

def SaveFigure(fig,plotType, fileName):
    plotFileName = f"{fileName}_{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}.jpg"
    plottingFileName = GetPlottingDirectory(plotFileName, plotType)
    fig.savefig(plottingFileName, dpi=300, bbox_inches='tight')

In [ ]:
#IMPORT CLASSES (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [ ]:
#IMPORT FUNCTIONS (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"Variable_Calculation"))
from FUNCTIONS_Variable_Calculation import *

In [ ]:
#IMPORT FUNCTIONS

import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
####################################
#LOADING CLASSES

In [ ]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="EntrainmentCalculation", dataName="EntrainmentCalculation",
                                dtype='int32')

In [ ]:
####################################
#FUNCTIONS

In [ ]:
#SETTING UP RUN

def LoadMeanLFC():
    # dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    # in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    # with open(in_file, 'rb') as f:
    #     MeanLFC = pickle.load(f)
    MeanLFC=2 #*#*
    return MeanLFC
MeanLFC=LoadMeanLFC()
print(f"Mean LFC is: {MeanLFC}\n")


def LoadAllCloudBase():
    # dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    # in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    # with open(in_file, 'rb') as f:
    #     all_cloudbase = pickle.load(f)
    all_cloudbase=1 #*#*
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
cloudbase=min_all_cloudbase
print(f"Minimum Cloudbase is: {cloudbase}\n")

In [ ]:
#APPLYING ENTRAINMENT CONSTANT
def ApplyEntrainmentConstant(arrayDictionary):
    """
    Multiply each array in the input dictionary by the 1D entrainment constant profile.
    Returns the processed arrays in the same order as the input dictionary.
    """
    entrainmentConstant = DataManager.LoadCalculations(
        DataManager.outputDataDirectory,
        dataName="EntrainmentConstant",
        verbose=False,
    )["entrainmentConstant"]

    outputDictionary = {}
    for var_name, arr in arrayDictionary.items():
        outputDictionary[var_name] = arr * entrainmentConstant[:, np.newaxis, np.newaxis]

    # Return arrays in the same order as input
    return tuple(outputDictionary[var_name] for var_name in arrayDictionary.keys())

# def ApplyEntrainmentConstant(arrayDictionary, vertical_axis=None):
#     """
#     Multiply each array in the input dictionary by the 1D entrainment constant profile.
#     Automatically detects which axis corresponds to the vertical dimension.
#     """
#     # Load entrainment constant
#     entrainmentConstant = DataManager.LoadCalculations(
#         DataManager.outputDataDirectory,
#         dataName="EntrainmentConstant",
#         verbose=False
#     )["entrainmentConstant"]
#     entrainmentConstant = np.asarray(entrainmentConstant)

#     outputDictionary = {}

#     for var_name, arr in arrayDictionary.items():
#         arr = np.asarray(arr)

#         # --- detect vertical axis automatically ---
#         if vertical_axis is None:
#             matches = np.where(np.array(arr.shape) == entrainmentConstant.size)[0]
#             if len(matches) == 0:
#                 raise ValueError(f"No dimension in '{var_name}' matches entrainment constant length {entrainmentConstant.size}")
#             vertical_axis_local = matches[0]
#         else:
#             vertical_axis_local = vertical_axis

#         # --- build correct broadcast shape ---
#         shape = [1] * arr.ndim
#         shape[vertical_axis_local] = entrainmentConstant.size
#         entrainmentConstant_reshaped = entrainmentConstant.reshape(shape)

#         # --- multiply along vertical axis ---
#         outputDictionary[var_name] = arr * entrainmentConstant_reshaped

#         # Debug info
#         # print(f"{var_name}: arr={arr.shape}, const={entrainmentConstant_reshaped.shape}, vertical_axis={vertical_axis_local}")

#     return tuple(outputDictionary[var_name] for var_name in arrayDictionary.keys())


In [ ]:
#CALCULATING ENTRAINMENT #*#*#*

PROCESSING=False #*#*
# PROCESSING=True

# def GetEntrainment_T(t,varName):
#     timeString=ModelData.timeStrings[t]
#     output = CallVariable(ModelData, DataManager, timeString, variableName=varName)
#     return output

def GetEntrainment_T(t, varName, cache=None):
    """Load variable for a given timestep, using cached open HDF5 files if available."""
    if cache is None:
        cache = {}

    timeString = ModelData.timeStrings[t]

    # If file for this timestep already in cache, reuse it
    if timeString in cache:
        f = cache[timeString]
    else:
        # Otherwise, open new file and cache it
        f = DataManager.GetTimestepData_V2(DataManager.inputDataDirectory, timeString)
        cache[timeString] = f

    # Load the desired variable from the open file (lazy read)
    if varName in f.keys():
        output = f[varName][:]  # read actual data
    else:
        # fallback for derived variables
        output = CallVariable(ModelData, DataManager, timeString, variableName=varName)
    return output
    
def MeanOut_YX(output):
    output_mean = np.mean(output,axis=(1,2))
    return output_mean

def MeanOut_ZY(output):
    output_mean = np.mean(output,axis=(0,1))
    return output_mean

def MeanOut_ZX(output):
    output_mean = np.mean(output,axis=(0,2))
    return output_mean

def GetEntrainment_Mean(varNames, meantype='TZ'):
    #defining mean function
    if meantype=='TZ':
        MeanOut = MeanOut_YX
        dim1=ModelData.Ntime
        dim2=ModelData.Nzh
    elif meantype=='TY':
        MeanOut = MeanOut_ZX
        dim1=ModelData.Ntime
        dim2=ModelData.Nyh
    elif meantype=='TX':
        MeanOut = MeanOut_ZY
        dim1=ModelData.Ntime
        dim2=ModelData.Nxh
        
    outputDict = {name: np.zeros((dim1, dim2), dtype=np.float32) for name in varNames}
    loaded_cache = {} #cache for loaded datasets
    try:
        for t in range(ModelData.Ntime):
            if np.mod(t,10)==0: print(f'Current time {t}')
    
            for varName in varNames:
                if 'Detrainment' in varName and t == ModelData.Ntime-1:
                    continue
                else:
                    variable = GetEntrainment_T(t,varName,cache=loaded_cache)
                    (variable,) = ApplyEntrainmentConstant({"variable": variable})
                    variableMean = MeanOut(variable)
                    outputDict[varName][t] = variableMean

    finally:
        # Closing all h5py handles
        for key, f in loaded_cache.items():
            try:
                f.close()
            except Exception as e:
                print(f"Warning: failed to close file {key}: {e}")

    return outputDict


def GetEntrainment_MeanZX(varNames, meantype='ZX'):
    """
    Compute the mean over time and Y for each variable in varNames.
    Returns a dictionary with shape (zh, xh) for each variable.
    """
    outputDict = {name: np.zeros((ModelData.Nzh, ModelData.Nxh), dtype=np.float32) for name in varNames}
    loaded_cache = {}

    try:
        for t in range(ModelData.Ntime):
            if np.mod(t, 10) == 0:
                print(f'Current time {t}')

            for varName in varNames:
                if 'Detrainment' in varName and t == ModelData.Ntime - 1:
                    continue
                else:
                    variable = GetEntrainment_T(t, varName, cache=loaded_cache)
                    (variable,) = ApplyEntrainmentConstant({"variable": variable})

                    # Average over y-axis (axis=2), result shape: (zh, xh)
                    yz_mean = np.mean(variable, axis=1)

                    # Accumulate over time
                    outputDict[varName] += yz_mean

        # Final average over time
        for varName in varNames:
            # Detrainment is one timestep shorter
            nt = ModelData.Ntime - 1 if 'Detrainment' in varName else ModelData.Ntime
            outputDict[varName] /= nt

    finally:
        # Close all open HDF5 handles
        for key, f in loaded_cache.items():
            try:
                f.close()
            except Exception as e:
                print(f"Warning: failed to close file {key}: {e}")

    return outputDict


In [ ]:
####################################
#CALCULATION FUNCTIONS

In [ ]:
#entrainmentdetrainment

def Calculate_EntrainmentDetrainment_Mean(meantype='TZ'):  
    if meantype=='ZX':
        GetEntrainment_Mean_Function = GetEntrainment_MeanZX
    else:
        GetEntrainment_Mean_Function = GetEntrainment_Mean
        
    #loading/calculation data
    try:
        # Try loading existing calculations
        arrayDictionary = DataManager.LoadCalculations(
            DataManager.outputDataDirectory,
            dataName=f"EntrainmentDetrainment_Mean{meantype}"
        )
        print(arrayDictionary.keys())
    except Exception as e:
        print("Loading failed. Running calculations and saving.")
        print("Error:", e)

        varNames = ["Entrainment_g","Entrainment_c",
                    "Detrainment_g","Detrainment_c"]
        arrayDictionary = GetEntrainment_Mean_Function(varNames,meantype)
    
        #SAVING
        DataManager.SaveCalculations(DataManager.outputDataDirectory, arrayDictionary, dataName=f"EntrainmentDetrainment_Mean{meantype}",dtype="float32")

    profile_array_e_g, profile_array_e_c, profile_array_d_g, profile_array_d_c = (
        arrayDictionary[k] for k in ["Entrainment_g", "Entrainment_c", "Detrainment_g", "Detrainment_c"]
    )
    return [profile_array_e_g,
            profile_array_e_c,
            profile_array_d_g,
            profile_array_d_c]

#entrainmentdetrainment

def Calculate_TransferEntrainmentDetrainment_Mean(meantype='TZ'):      
    if meantype=='ZX':
        GetEntrainment_Mean_Function = GetEntrainment_MeanTZ
    else:
        GetEntrainment_Mean_Function = GetEntrainment_Mean
        
    #loading/calculation data
    try:
        # Try loading existing calculations
        arrayDictionary = DataManager.LoadCalculations(
            DataManager.outputDataDirectory,
            dataName=f"TransferEntrainmentDetrainment_Mean{meantype}"
        )
    except Exception as e:
        print("Loading failed. Running calculations and saving.")
        print("Error:", e)

        varNames = ["TransferEntrainment_c_to_g","TransferEntrainment_g_to_c",
                    "TransferDetrainment_c_to_g","TransferDetrainment_g_to_c"]
        arrayDictionary = GetEntrainment_Mean_Function(varNames,meantype)
    
        #SAVING
        DataManager.SaveCalculations(DataManager.outputDataDirectory, arrayDictionary, dataName=f"TransferEntrainmentDetrainment_Mean{meantype}",dtype="float32")

    TransferEntrainment_c_to_g, TransferEntrainment_g_to_c, TransferDetrainment_c_to_g, TransferDetrainment_g_to_c = (
        arrayDictionary[k] for k in ["TransferEntrainment_c_to_g", "TransferEntrainment_g_to_c", "TransferDetrainment_c_to_g", "TransferDetrainment_g_to_c"]
    )

    return [TransferEntrainment_c_to_g,
            TransferEntrainment_g_to_c,
            TransferDetrainment_c_to_g,
            TransferDetrainment_g_to_c]

In [ ]:
####################################
#CALCULATION

In [ ]:
#entrainmentdetrainment
[profile_array_e_g_MeanTZ,
 profile_array_e_c_MeanTZ,
 profile_array_d_g_MeanTZ,
 profile_array_d_c_MeanTZ] = Calculate_EntrainmentDetrainment_Mean(meantype='TZ')

#transfer entrainmentdetrainment
[profile_array_c_to_g_E_MeanTZ,
 profile_array_g_to_c_E_MeanTZ,
 profile_array_c_to_g_D_MeanTZ,
 profile_array_g_to_c_D_MeanTZ] = Calculate_TransferEntrainmentDetrainment_Mean(meantype='TZ')

In [ ]:
#entrainmentdetrainment
[profile_array_e_g_MeanTY,
 profile_array_e_c_MeanTY,
 profile_array_d_g_MeanTY,
 profile_array_d_c_MeanTY] = Calculate_EntrainmentDetrainment_Mean(meantype='TY')

In [ ]:
#entrainmentdetrainment
[profile_array_e_g_MeanTX,
 profile_array_e_c_MeanTX,
 profile_array_d_g_MeanTX,
 profile_array_d_c_MeanTX] = Calculate_EntrainmentDetrainment_Mean(meantype='TX')

In [ ]:
#entrainmentdetrainment
[profile_array_e_g_MeanZX,
 profile_array_e_c_MeanZX,
 profile_array_d_g_MeanZX,
 profile_array_d_c_MeanZX] = Calculate_EntrainmentDetrainment_Mean(meantype='ZX')

In [ ]:
####################################
#PLOTTING FUNCTIONS

In [ ]:
def GetData_Mean(type,meantype='TZ'):
    if type=='general':
        profile_array_e=globals()[f"profile_array_e_g_Mean{meantype}"]
        profile_array_d=-globals()[f"profile_array_d_g_Mean{meantype}"]
        profile_array_net=profile_array_e-profile_array_d
    if type=='cloudy':
        profile_array_e=globals()[f"profile_array_e_c_Mean{meantype}"]
        profile_array_d=-globals()[f"profile_array_d_c_Mean{meantype}"]
        profile_array_net=profile_array_e-profile_array_d

    return profile_array_e,profile_array_d,profile_array_net

def GetTransferData_Mean(meantype='TZ'):
    profile_array_c_to_g_E=globals()[f"profile_array_c_to_g_E_Mean{meantype}"]
    profile_array_g_to_c_E=globals()[f"profile_array_g_to_c_E_Mean{meantype}"]
    profile_array_c_to_g_D=globals()[f"profile_array_c_to_g_D_Mean{meantype}"]
    profile_array_g_to_c_D=globals()[f"profile_array_g_to_c_D_Mean{meantype}"]
    return profile_array_c_to_g_E,profile_array_g_to_c_E,profile_array_c_to_g_D,profile_array_g_to_c_D

In [ ]:
def plot_mean_entrainment(ax, profile_array_e, profile_array_d, title, linestyle='solid'):
    zh=ModelData.zh

    # Compute mean profiles
    e = np.mean(profile_array_e, axis=0)
    d = np.mean(profile_array_d, axis=0)
    net = np.mean(profile_array_e - profile_array_d, axis=0)

    # Plot
    ax.plot(e, zh, linestyle=linestyle, color='blue', label='Entrainment')
    ax.plot(d, zh, linestyle=linestyle, color='red', label='Detrainment')
    ax.plot(net, zh, linestyle=linestyle, color='black', label='Net Entrainment')
    ax.axvline(0, color='gray', linestyle='dashed',zorder=-10)

    ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
    ax.axhline(MeanLFC, color='green', linestyle='dashed', lw=1.2)

    ax.grid(True)
    ax.set_title(f"{title}",fontsize=10.5)
    ax.set_xlabel(r"($kg\ m^{-3}\ s^{-1}$)")  
    ax.set_ylabel('z (km)')
    ax.set_ylim(bottom=0)
    ax.legend(loc='upper right')
    ax.set_ylim(0,20)

    #FIXED TICKS
    SnapLimitsToTicks([ax], dim='x')

    
    # Format x-axis in scientific notation
    apply_scientific_notation([ax],decimals=2)

In [ ]:
def compute_means_for_transfer(c_to_g_E, g_to_c_E, c_to_g_D, g_to_c_D,
                               profile_array_e_g, profile_array_e_c,
                               profile_array_d_g, profile_array_d_c):
    """Compute and return mean profiles for transfer rates and entrainment/detrainment arrays."""

    # Compute means of transfer rates
    c_to_g_E_mean = np.mean(c_to_g_E, axis=0)
    g_to_c_E_mean = np.mean(g_to_c_E, axis=0)
    c_to_g_D_mean = np.mean(c_to_g_D, axis=0)
    g_to_c_D_mean = np.mean(g_to_c_D, axis=0)

    # Compute means of entrainment/detrainment profiles
    mean_e_g = np.mean(profile_array_e_g, axis=0)
    mean_e_c = np.mean(profile_array_e_c, axis=0)
    mean_d_g = np.mean(profile_array_d_g, axis=0)
    mean_d_c = np.mean(profile_array_d_c, axis=0)

    return {
        'c_to_g_E_mean': c_to_g_E_mean,
        'g_to_c_E_mean': g_to_c_E_mean,
        'c_to_g_D_mean': c_to_g_D_mean,
        'g_to_c_D_mean': g_to_c_D_mean,
        'mean_e_g': mean_e_g,
        'mean_e_c': mean_e_c,
        'mean_d_g': mean_d_g,
        'mean_d_c': mean_d_c
    }

    
def plot_transfer_rate(ax, means, title):
    zh = ModelData.zh

    c_to_g_mean_E = means['c_to_g_E_mean']
    g_to_c_mean_E = means['g_to_c_E_mean']
    c_to_g_mean_D = means['c_to_g_D_mean']
    g_to_c_mean_D = means['g_to_c_D_mean']

    # ax.plot(g_to_c_mean_E, zh, color='blue', label='General → Cloudy',linestyle='solid')
    # ax.plot(c_to_g_mean_E, zh, color='red', label='Cloudy → General',linestyle='solid')
    ax.plot(g_to_c_mean_E, zh, color='blue', label='General → Cloudy (Entrainment)',linestyle='solid')
    ax.plot(c_to_g_mean_E, zh, color='red', label='Cloudy → General (Entrainment)',linestyle='solid')
    ax.plot(c_to_g_mean_D, zh, color='blue', label='Cloudy → General (Detrainment)',linestyle='dashed')
    ax.plot(g_to_c_mean_D, zh, color='red', label='General → Cloudy (Detrainment)',linestyle='dashed')
    ax.axvline(0, color='black', linewidth=1)

    ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
    ax.axhline(MeanLFC, color='forestgreen', linestyle='dashed', lw=1.2)

    ax.grid(True)
    ax.set_title(f"{title}")
    ax.set_xlabel('Mass Transfer Rate')
    ax.set_xlabel(r"($kg m^{-3} s^{-1}$)")  
    ax.set_ylabel('z (km)')
    ax.set_ylim(bottom=0,top=20)
    ax.legend()
    
    SnapLimitsToTicks([ax], dim='x')
    apply_scientific_notation([ax],decimals=2)

def plot_transfer_ratio(ax, means, title):
    zh = ModelData.zh

    mean_e_g = means['mean_e_g']
    mean_e_c = means['mean_e_c']
    mean_d_g = means['mean_d_g']
    mean_d_c = means['mean_d_c']
    
    mean_c_to_g_E = means['c_to_g_E_mean']
    mean_g_to_c_E = means['g_to_c_E_mean']
    mean_c_to_g_D = means['c_to_g_D_mean']
    mean_g_to_c_D = means['g_to_c_D_mean']

    threshold = 0
    with np.errstate(divide='ignore', invalid='ignore'):
        ratio_1 = np.where(mean_e_c > threshold, mean_g_to_c_E / mean_e_c, np.nan)
        ratio_2 = np.where(mean_d_g > threshold, mean_g_to_c_D / mean_d_g, np.nan)
        ratio_3 = np.where(mean_e_g > threshold, mean_c_to_g_E / mean_e_g, np.nan)
        ratio_4 = np.where(mean_d_c > threshold, mean_c_to_g_D / mean_d_c, np.nan)

    # print(np.nanmean(ratio_1))
    # print(np.nanmean(ratio_2))
    # print(np.nanmean(ratio_3))
    # print(np.nanmean(ratio_4))

    ax.plot(ratio_1*100, zh, color='blue', label='General → Cloudy / Cloudy Entrainment')
    ax.plot(ratio_2*100, zh, color='deepskyblue', label='General → Cloudy / General Detrainment')
    ax.plot(ratio_3*100, zh, color='red', label='Cloudy → General / General Entrainment')
    ax.plot(ratio_4*100, zh, color='orangered', label='Cloudy → General / Cloudy Detrainment')

    ax.axvline(0, color='black', linestyle='dashed', linewidth=1)
    # ax.axvline(1, color='black', linestyle='dashed', linewidth=1)
    ax.axvline(100, color='black', linestyle='dashed', linewidth=1)
    ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
    ax.axhline(MeanLFC, color='green', linestyle='dashed', lw=1.2)

    ax.grid(True)
    ax.set_title(f"{title}")
    # ax.set_xlabel('Ratio')
    ax.set_xlabel('%')
    ax.set_ylabel('z (km)')
    # ax.set_xlim(-0.05, 1.05)
    ax.set_xlim(-5, 105)
    ax.set_ylim(bottom=0,top=20)
    ax.legend(fontsize=10.5-2, loc='upper right')
    # apply_scientific_notation([ax])


In [ ]:
def Plot_EntrainmentDetrainmentRows_Mean(array_triplets, type_labels, plotting=True, meantype='TZ'):
    if not plotting:
        return
        
    cmap1 = plt.cm.viridis
    cmap2 = plt.cm.seismic 
    n_levels = 29

    num_rows = len(array_triplets)

    fig = plt.figure(figsize=(18, 4 * num_rows))
    gs = GridSpec(num_rows, 4, figure=fig)
    all_ax1, all_ax2, all_ax3 = [], [], []
    
    for row_idx, ((profile_array_e, profile_array_d, profile_array_net), type_label) in enumerate(zip(array_triplets, type_labels)):
        # vmax_shared = np.max([np.max(profile_array_e), np.max(profile_array_d)])
        # norm_shared = mcolors.Normalize(vmin=0, vmax=vmax_shared)
        norm_shared = None

        if meantype == 'TZ':
            x = np.arange(profile_array_e.shape[0])
            x_label = "t (hours)"

            y = ModelData.zh
            y_label = "z (km)"
            y_lim = (np.min(ModelData.zh), np.max(ModelData.zh))
            # y_lim = tuple(np.round(y_lim, 0))

        if meantype == 'TY':
            x = np.arange(profile_array_e.shape[0])
            x_label = "t (hours)"
            y = ModelData.yh
            y_label = "y (km)"
            y_lim = (np.min(ModelData.yh), np.max(ModelData.yh))
            # y_lim = tuple(np.round(y_lim, 0))
        elif meantype == 'TX':
            x = np.arange(profile_array_e.shape[0])
            x_label = "t (hours)"
            y = ModelData.xh
            y_label = "x (km)"
            y_lim = (np.min(ModelData.xh), np.max(ModelData.xh))
            # y_lim = tuple(np.round(y_lim, 0))
        elif meantype == 'ZX':
            x = ModelData.xh
            x_label = "x (km)"
            y = ModelData.zh
            y_label = "z (km)"
            y_lim = (np.min(ModelData.zh), np.max(ModelData.zh))
            # y_lim = tuple(np.round(y_lim, 0))        
        mins=(ModelData.time[1].astype(int)/1e9/60)
        xTickInterval = 60/mins
        plot_kwargs = {
            'PlotData': None,
            'xTickLabels': x, 'yTickLabels': y,
            'contour_type': 'line',#'fill',
            # 'contour_type': 'fill',
            'num_xticks': 11, 'round_xticks': None, 'xTickInterval': xTickInterval,
            'num_yticks': 15, 'round_yticks': 2, 'yTickInterval': None,
            'add_colorbar': True, 'fig': fig, 'levels': 29, 'colorbar_label_rotation': 0, 'colorbar_label': None,
            'xlabel': x_label, 'ylabel': y_label,
            'solid_contour_labels': False, 'solid_contour_round': None,
            'xtick_rotation': 0, 'ytick_rotation': 0, 'cbar_rotation': 0,
            'save_path': None, 'save_dpi': 300,
            'colorbar_kwargs': {
                'extend': 'both'
            },
            'norm': norm_shared
        }

        ax1 = fig.add_subplot(gs[row_idx, 0])
        ax2 = fig.add_subplot(gs[row_idx, 1])
        ax3 = fig.add_subplot(gs[row_idx, 2])

        all_ax1.append(ax1)
        all_ax2.append(ax2)
        all_ax3.append(ax3)

        plot_kwargs1 = plot_kwargs.copy()
        plot_kwargs1['PlotData'] = profile_array_e.copy().T
        plot_kwargs1['cmap'] = cmap1
        [contour1, cbar1] = UltimateContourPlot(ax1, **plot_kwargs1)
        ax1.set_ylim(y_lim)
        ax1.set_title(f'Entrainment {type_label}',fontsize=8)

        plot_kwargs2 = plot_kwargs.copy()
        plot_kwargs2['PlotData'] = profile_array_d.copy().T
        plot_kwargs2['cmap'] = cmap1
        [contour2, cbar2] = UltimateContourPlot(ax2, **plot_kwargs2)
        ax2.set_ylim(y_lim)
        ax2.set_title(f'Detrainment {type_label}',fontsize=8)

        plot_data3 = profile_array_net.copy().T

        #FIXING COLORBAR NORM
        #####################   
        # vmin = -np.max(abs(profile_array_net)) / 2
        # vmax = +np.max(abs(profile_array_net))
        # levels = np.linspace(vmin, vmax, n_levels)
        # norm = mcolors.BoundaryNorm(boundaries=levels, ncolors=256)
            
        vmin = np.min(profile_array_net)
        vmax = np.max(profile_array_net)
        vcenter = 0.0
        levels = np.linspace(vmin, vmax, n_levels)
        norm = TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)
        #####################
        
        plot_kwargs3 = plot_kwargs.copy()
        plot_kwargs3['PlotData'] = plot_data3
        plot_kwargs3['cmap'] = cmap2
        plot_kwargs3['norm'] = norm
        plot_kwargs3['levels'] = levels
        [contour3, cbar3] = UltimateContourPlot(ax3, **plot_kwargs3)
        ax3.set_ylim(y_lim)
        ax3.set_title(f'Net Entrainment {type_label}',fontsize=8)
        # fig.suptitle(f"{type_labels[row_idx][0].upper() + type_labels[row_idx][1:]} Updraft Entrainment/Detrainment")

        def apply_scientific_notation_colorbar(cbars):
            from matplotlib.ticker import ScalarFormatter
            formatter = ScalarFormatter(useMathText=True)
            formatter.set_powerlimits((-2, 2))
            for cbar in cbars:
                cbar.formatter = formatter
                cbar.update_ticks()

        apply_scientific_notation_colorbar([cbar1, cbar2, cbar3])

        #FIXING CONTOUR COLORBAR LINES THICKNESS
        for cbar in [cbar1, cbar2, cbar3]:
            for cbar_line in cbar.ax.collections:
                cbar_line.set_linewidth(3)
        if meantype=='TZ':
            for ax in [ax1,ax2,ax3]:
                ax.axhline(cloudbase, color='purple', linestyle='dashed',lw=1.2)
                ax.axhline(MeanLFC, color='green', linestyle='dashed',lw=1.2)
        if meantype!='ZX':
            for ax in [ax1,ax2,ax3]:
                xticks = ax.get_xticks()
                new_labels = [f"{tick*mins/(60)+6:.0f}" for tick in xticks]
                ax.set_xticklabels(new_labels)
        if meantype == 'ZX':
            num_xticks = 4
            x_indices = np.linspace(0, len(x)-1, num_xticks, dtype=int)
            x_ticks = x[x_indices]
            for ax in [ax1, ax2, ax3]:
                ax.set_xticks(x_ticks)
                ax.set_xticklabels([f"{val:.1f}" for val in x_ticks])
            
    ticks = np.linspace(y_lim[0], y_lim[1], num=11)
    for ax in all_ax1 + all_ax2 + all_ax3:# + [ax4, ax5]:
        ax.set_ylim(y_lim)
        ax.set_yticks(ticks)
        ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
        ax.margins(y=0)

    if meantype=='TZ':
        ax4 = fig.add_subplot(gs[0, 3])
        ax5 = fig.add_subplot(gs[1, 3])
        plot_mean_entrainment(ax4, profile_array_e1, profile_array_d1, title='Mean Vertical Profile')
        plot_mean_entrainment(ax5, profile_array_e2, profile_array_d2, title='Mean Vertical Profile')

    #SPACING
    fig.subplots_adjust(top=0.92, wspace=0.4, hspace=0.4)
    
    return fig

In [ ]:
####################################
#PLOTTING

In [ ]:
meantype='TZ'

#LOAD DATA
type1='general';type2='cloudy'
[profile_array_e1,profile_array_d1,profile_array_net1]=GetData_Mean(type=type1,meantype=meantype)
[profile_array_e2,profile_array_d2,profile_array_net2]=GetData_Mean(type=type2,meantype=meantype)

#CONTOUR PLOTS
array_triplets = [
    (profile_array_e1, profile_array_d1, profile_array_net1),
    (profile_array_e2, profile_array_d2, profile_array_net2)
]
type_labels = ['(General Updraft)', '(Cloudy Updraft)']

fig = Plot_EntrainmentDetrainmentRows_Mean(array_triplets, type_labels, meantype=meantype)

#SAVING 
fileName=f"EntrainmentDetrainment_MeanTZ" 
SaveFigure(fig,plotType="Variable_Calculation/EntrainmentCalculation",fileName=fileName)

In [ ]:
meantype='TY'

#LOAD DATA
type1='general';type2='cloudy'
[profile_array_e1,profile_array_d1,profile_array_net1]=GetData_Mean(type=type1,meantype=meantype)
[profile_array_e2,profile_array_d2,profile_array_net2]=GetData_Mean(type=type2,meantype=meantype)

#CONTOUR PLOTS
array_triplets = [
    (profile_array_e1, profile_array_d1, profile_array_net1),
    (profile_array_e2, profile_array_d2, profile_array_net2)
]
type_labels = ['(General Updraft)', '(Cloudy Updraft)']

fig = Plot_EntrainmentDetrainmentRows_Mean(array_triplets, type_labels, meantype=meantype)

#SAVING 
fileName=f"EntrainmentDetrainment_MeanTY" 
SaveFigure(fig,plotType="Variable_Calculation/EntrainmentCalculation",fileName=fileName)

In [ ]:
meantype='TX'

#LOAD DATA
type1='general';type2='cloudy'
[profile_array_e1,profile_array_d1,profile_array_net1]=GetData_Mean(type=type1,meantype=meantype)
[profile_array_e2,profile_array_d2,profile_array_net2]=GetData_Mean(type=type2,meantype=meantype)

#CONTOUR PLOTS
array_triplets = [
    (profile_array_e1, profile_array_d1, profile_array_net1),
    (profile_array_e2, profile_array_d2, profile_array_net2)
]
type_labels = ['(General Updraft)', '(Cloudy Updraft)']

fig = Plot_EntrainmentDetrainmentRows_Mean(array_triplets, type_labels, meantype=meantype)

#SAVING 
fileName=f"EntrainmentDetrainment_MeanTX" 
SaveFigure(fig,plotType="Variable_Calculation/EntrainmentCalculation",fileName=fileName)

In [ ]:
meantype='ZX'

#LOAD DATA
type1='general';type2='cloudy'
[profile_array_e1,profile_array_d1,profile_array_net1]=GetData_Mean(type=type1,meantype=meantype)
[profile_array_e2,profile_array_d2,profile_array_net2]=GetData_Mean(type=type2,meantype=meantype)

#CONTOUR PLOTS
array_triplets = [
    (profile_array_e1.T, profile_array_d1.T, profile_array_net1.T),
    (profile_array_e2.T, profile_array_d2.T, profile_array_net2.T)
]
type_labels = ['(General Updraft)', '(Cloudy Updraft)']

fig = Plot_EntrainmentDetrainmentRows_Mean(array_triplets, type_labels, meantype=meantype)

#SAVING 
fileName=f"EntrainmentDetrainment_MeanZX" 
SaveFigure(fig,plotType="Variable_Calculation/EntrainmentCalculation",fileName=fileName)

In [ ]:
#COMBINED ENTRAINMENT

In [ ]:
profile_array_e_g_MeanTZ.shape

In [ ]:
# === Compute means for transfer rate and ratio plots ===
meantype='TZ'
#LOAD DATA
type1='general';type2='cloudy'
[profile_array_e1,profile_array_d1,profile_array_net1]=GetData_Mean(type=type1,meantype=meantype)
[profile_array_e2,profile_array_d2,profile_array_net2]=GetData_Mean(type=type2,meantype=meantype)

profile_array_c_to_g_E,profile_array_g_to_c_E,profile_array_c_to_g_D,profile_array_g_to_c_D = GetTransferData_Mean()
    
means = compute_means_for_transfer(
    c_to_g_E=profile_array_c_to_g_E,
    g_to_c_E=profile_array_g_to_c_E,
    c_to_g_D=profile_array_c_to_g_D,
    g_to_c_D=profile_array_g_to_c_D,
    profile_array_e_g=profile_array_e_g_MeanTZ,
    profile_array_e_c=profile_array_e_c_MeanTZ,
    profile_array_d_g=profile_array_d_g_MeanTZ,
    profile_array_d_c=profile_array_d_c_MeanTZ
)

# === Set up figure and subplots ===
fig = plt.figure(figsize=(18, 6))
gs = gridspec.GridSpec(1, 4, figure=fig, wspace=0.3, )


ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax3 = fig.add_subplot(gs[2])
ax4 = fig.add_subplot(gs[3])

# === Plot each panel ===
plot_mean_entrainment(ax1, profile_array_e1, profile_array_d1, title='')
plot_mean_entrainment(ax2, profile_array_e2, profile_array_d2, title='')
plot_transfer_rate(ax3, means, title='')
plot_transfer_ratio(ax4, means, title='')
fig.subplots_adjust(top=0.92, wspace=0.4, hspace=0.4)

#SAVING 
fileName=f"TransferEntrainmentDetrainment" 
SaveFigure(fig,plotType="Variable_Calculation/EntrainmentCalculation",fileName=fileName)